**Install the required solver**

In [ ]:
!pip install gurobipy  # install gurobipy, if not already installed
!pip install folium
!pip install pandas

**Loading patient-donor pairs and transplant center data**

In [ ]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')
pair_data_path = "/content/drive/MyDrive/6.C57 Final Project/Data/pair_small.csv"
center_data_path = "/content/drive/MyDrive/6.C57 Final Project/Data/center_small.csv"

# Load data from CSV files
# pair_data_path = "./data/pair_small.csv"
# center_data_path = "./data/center_small.csv"

# Load pair data
pairs = pd.read_csv(pair_data_path)

# Load transplant center data
centers = pd.read_csv(center_data_path)

print("pairs", pairs)
print("centers", centers)

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/6.C57 Final Project/Data/pair_small.csv'

**Basic Model (where all pairs are satisfied)**



In [ ]:
from gurobipy import Model, GRB, quicksum

# Minimal data for testing
n = 3  # Number of donor-patient pairs
# Compatibility matrix (1 if compatible, 0 otherwise)
c = [[0, 1, 0],
     [0, 0, 1],
     [1, 0, 0]]

# Create a model
model = Model("Kidney_Exchange")

# Decision variables: y[i][j] = 1 if donor i donates to patient j, 0 otherwise
y = model.addVars(n, n, vtype=GRB.BINARY, name="y")

# Objective: Maximize the number of transplants
model.setObjective(quicksum(y[i, j] for i in range(n) for j in range(n)), GRB.MAXIMIZE)

# Constraints

# Compatibility constraint: y_ij <= c_ij
for i in range(n):
    for j in range(n):
        model.addConstr(y[i, j] <= c[i][j], name=f"compatibility_{i}_{j}")

# Each donor can donate to at most one patient: Σ y_ij <= 1, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) <= 1, name=f"donate_limit_{i}")

# Each patient can receive a kidney from at most one donor: Σ y_ij <= 1, ∀ j
for j in range(n):
    model.addConstr(quicksum(y[i, j] for i in range(n)) <= 1, name=f"receive_limit_{j}")

# Reciprocal donation constraint: Σ y_ij = Σ y_ji, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) == quicksum(y[j, i] for j in range(n)),
                    name=f"reciprocal_donation_{i}")

# Optimize the model
model.optimize()

print()
print("Results: ")
# Print the results
transplants = []
if model.status == GRB.OPTIMAL:
    for i in range(n):
        for j in range(n):
            if y[i, j].x > 0.5:  # Only consider y[i,j] that are set to 1
                print(f"Donor {i} donates to Patient {j}")
                transplants.append((i, j))

print("Optimal transplants:", transplants)

**Basic model (where no one is satisfied)**

In [ ]:
# Minimal data for testing
n = 3  # Number of donor-patient pairs
# Compatibility matrix (1 if compatible, 0 otherwise)
c = [[0, 1, 0],
     [0, 0, 1],
     [0, 0, 0]]

# Create a model
model = Model("Kidney_Exchange")

# Decision variables: y[i][j] = 1 if donor i donates to patient j, 0 otherwise
y = model.addVars(n, n, vtype=GRB.BINARY, name="y")

# Objective: Maximize the number of transplants
model.setObjective(quicksum(y[i, j] for i in range(n) for j in range(n)), GRB.MAXIMIZE)

# Constraints

# Compatibility constraint: y_ij <= c_ij
for i in range(n):
    for j in range(n):
        model.addConstr(y[i, j] <= c[i][j], name=f"compatibility_{i}_{j}")

# Each donor can donate to at most one patient: Σ y_ij <= 1, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) <= 1, name=f"donate_limit_{i}")

# Each patient can receive a kidney from at most one donor: Σ y_ij <= 1, ∀ j
for j in range(n):
    model.addConstr(quicksum(y[i, j] for i in range(n)) <= 1, name=f"receive_limit_{j}")

# Reciprocal donation constraint: Σ y_ij = Σ y_ji, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) == quicksum(y[j, i] for j in range(n)),
                    name=f"reciprocal_donation_{i}")

# Optimize the model
model.optimize()

print()
print("Results: ")
# Print the results
transplants = []
if model.status == GRB.OPTIMAL:
    for i in range(n):
        for j in range(n):
            if y[i, j].x > 0.5:  # Only consider y[i,j] that are set to 1
                print(f"Donor {i} donated to Patient {j}")
                transplants.append((i, j))

print("Optimal transplants:", transplants)

**Base model using data**

In [ ]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

# # Load data from CSV files
# pair_data_path = "./data/pair_small.csv"
# pairs = pd.read_csv(pair_data_path)

# Extract the number of pairs
n = len(pairs)

# Create a compatibility matrix based only on exact blood type match
c = [[0 for _ in range(n)] for _ in range(n)]

for i in range(n):
    for j in range(n):
        donor = pairs.iloc[i]["Donor"]
        patient = pairs.iloc[j]["Patient"]
        # Only exact blood type matches are considered compatible
        c[i][j] = 1 if donor == patient else 0

# Create the model
model = Model("Kidney_Exchange")

# Decision variables: y[i][j] = 1 if donor i donates to patient j, 0 otherwise
y = model.addVars(n, n, vtype=GRB.BINARY, name="y")

# Objective: Maximize the number of transplants
model.setObjective(quicksum(y[i, j] for i in range(n) for j in range(n)), GRB.MAXIMIZE)

# Constraints

# Compatibility constraint: y_ij <= c_ij
for i in range(n):
    for j in range(n):
        model.addConstr(y[i, j] <= c[i][j], name=f"compatibility_{i}_{j}")

# Each donor can donate to at most one patient: Σ y_ij <= 1, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) <= 1, name=f"donate_limit_{i}")

# Each patient can receive a kidney from at most one donor: Σ y_ij <= 1, ∀ j
for j in range(n):
    model.addConstr(quicksum(y[i, j] for i in range(n)) <= 1, name=f"receive_limit_{j}")

# Reciprocal donation constraint: Σ y_ij = Σ y_ji, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) == quicksum(y[j, i] for j in range(n)),
                    name=f"reciprocal_donation_{i}")

# Optimize the model
model.optimize()

# Print the results
print()
print("Results:")
transplants = []
if model.status == GRB.OPTIMAL:
    for i in range(n):
        for j in range(n):
            if y[i, j].x > 0.5:  # Only consider y[i,j] that are set to 1
                print(f"Donor {i} donates to Patient {j}")
                transplants.append((i, j))

print("Optimal transplants:", transplants)


**Model with Altruistic Donor (same as previous except adding altruistic donor makes every pair satisfied)**

In [ ]:
from gurobipy import Model, GRB, quicksum

# Minimal data for testing
n = 4  # Number of donor-patient pairs
# Compatibility matrix (1 if compatible, 0 otherwise)
c = [[0, 1, 0, 1],
     [0, 0, 1, 1],
     [0, 0, 0, 1],
     [1, 1, 0, 0]]

# Create a model
model = Model("Kidney_Exchange")

# Decision variables: y[i][j] = 1 if donor i donates to patient j, 0 otherwise
y = model.addVars(n, n, vtype=GRB.BINARY, name="y")

# Objective: Maximize the number of transplants
model.setObjective(quicksum(y[i, j] for i in range(n) for j in range(n)), GRB.MAXIMIZE)

# Constraints

# Compatibility constraint: y_ij <= c_ij
for i in range(n):
    for j in range(n):
        model.addConstr(y[i, j] <= c[i][j], name=f"compatibility_{i}_{j}")

# Each donor can donate to at most one patient: Σ y_ij <= 1, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) <= 1, name=f"donate_limit_{i}")

# Each patient can receive a kidney from at most one donor: Σ y_ij <= 1, ∀ j
for j in range(n):
    model.addConstr(quicksum(y[i, j] for i in range(n)) <= 1, name=f"receive_limit_{j}")

# Reciprocal donation constraint: Σ y_ij = Σ y_ji, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) == quicksum(y[j, i] for j in range(n)),
                    name=f"reciprocal_donation_{i}")

# Optimize the model
model.optimize()

print()
print("Results: ")
# Print the results
transplants = []
if model.status == GRB.OPTIMAL:
    for i in range(n):
        for j in range(n):
            if y[i, j].x > 0.5:  # Only consider y[i,j] that are set to 1
                print(f"Donor {i} donated to patient {j}")
                transplants.append((i, j))

print("Optimal transplants:", transplants)

**Model with Altruistic Donor using Data (same as previous except adding altruistic donor makes every pair satisfied)**




In [ ]:
# Number of donor-patient pairs
n = len(pairs)

# Create compatibility matrix (1 if compatible, 0 otherwise)
c = [[0 for _ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        donor = pairs.iloc[i]["Donor"]
        patient = pairs.iloc[j]["Patient"]
        c[i][j] = 1 if donor == patient else 0  # Exact matches only

# Altruistic donor list
altruistic_donors = pairs[pairs["Altruist"] == 1].index.tolist()

# Create a model
model = Model("Kidney_Exchange_With_Altruistic_Donors")

# Decision variables: y[i][j] = 1 if donor i donates to patient j, 0 otherwise
y = model.addVars(n, n, vtype=GRB.BINARY, name="y")

# Objective: Maximize the number of transplants
model.setObjective(quicksum(y[i, j] for i in range(n) for j in range(n)), GRB.MAXIMIZE)

# Constraints

# Compatibility constraint: y_ij <= c_ij
for i in range(n):
    for j in range(n):
        model.addConstr(y[i, j] <= c[i][j], name=f"compatibility_{i}_{j}")

# Each donor can donate to at most one patient: Σ y_ij <= 1, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) <= 1, name=f"donate_limit_{i}")

# Each patient can receive a kidney from at most one donor: Σ y_ij <= 1, ∀ j
for j in range(n):
    model.addConstr(quicksum(y[i, j] for i in range(n)) <= 1, name=f"receive_limit_{j}")

# Reciprocal donation constraint: Σ y_ij = Σ y_ji, ∀ i (except altruistic donors)
for i in range(n):
    if i not in altruistic_donors:
        model.addConstr(quicksum(y[i, j] for j in range(n)) == quicksum(y[j, i] for j in range(n)),
                        name=f"reciprocal_donation_{i}")

# Optimize the model
model.optimize()

# Print the results
print()
print("Results:")
transplants = []
if model.status == GRB.OPTIMAL:
    for i in range(n):
        for j in range(n):
            if y[i, j].x > 0.5:  # Match found
                print(f"Donor {i} donates to Patient {j}")
                transplants.append((i, j))

print("Optimal transplants:", transplants)


**Model with Capacity (using data) (Ashwini's version)**

In [ ]:
# # Load data from CSV files
# pair_data_path = "./data/pair_small.csv"
# center_data_path = "./data/center_small.csv"

# # Load pair data
# pairs = pd.read_csv(pair_data_path)

# # Load transplant center data
# centers = pd.read_csv(center_data_path)

print("pairs", pairs)
print("centers", centers)

# Number of pairs and centers
n = len(pairs)
m = len(centers)

# Load the data
# data/pair_small.csv
# each pair have a state with its latitute and longitute
# we should connect pairs together to form a meaningful transplant
# the donor for a pair should match the patient of the matching pair
# data structure:
'''
dataset_id,Pair,Patient,Donor,Wife-P?,%Pra,Out-Deg,Altruist,State,lat,lng
1,1,A,B,0,0.05,2,0,Illinois,39.798363,-89.654961
1,2,O,A,0,0.05,4,0,California,38.576668,-121.493629
1,3,A,B,0,0.05,2,0,Michigan,42.733635,-84.555328
1,4,O,A,1,0.5875,3,0,California,38.576668,-121.493629
1,5,B,AB,0,0.05,0,0,New Jersey,40.220596,-74.769913
'''


# data/transplant_center_dataset.csv
'''
hdg,mix-text_weightBold,Deceased Donor Transplants in a year,Living Donor Transplants in a year,Getting a Deceased Donor Transplant Faster,1-Year kidney Survival,lat,lng
Mayo Clinic Hospital Arizona,"Phoenix, AZ",381,121,4,3,33.6594309,-111.9563162
Tampa General Hospital,"Tampa, FL",356,93,1,3,27.937527,-82.4587507
University of California San Francisco Medical Center,"San Francisco, CA",314,117,3,4,37.7627945,-122.4580139
Saint Barnabas Medical Center,"Livingston, NJ",282,105,5,2,40.782424,-74.315139
The Cleveland Clinic Foundation,"Cleveland, OH",280,46,4,4,41.5031026,-81.6200078
'''

# Create compatibility matrix (1 if compatible, 0 otherwise)
c = [[0 for _ in range(n)] for _ in range(n)]
for i in range(n):
    for j in range(n):
        donor = pairs.iloc[i]["Donor"]
        patient = pairs.iloc[j]["Patient"]
        c[i][j] = 1 if donor == patient else 0  # Exact matches only

# Create a model
model = Model("Kidney_Exchange_Capacity")

# Decision variables: y[i][j][k] = 1 if donor i donates to patient j at transplant center k, 0 otherwise
y = model.addVars(n, n, m, vtype=GRB.BINARY, name="y")  # Donor-to-patient match

# Objective: Maximize the number of transplants
model.setObjective(
    quicksum(y[i, j, k] for i in range(n) for j in range(n) for k in range(m)),
    GRB.MAXIMIZE
)

# Another set of decision variables is c, which is a binary array with the length of the length of transplant center. it should only have one 1 as the selected transplant center

# Constraints
# Compatibility constraint: y_ij <= c_ij
for i in range(n):
    for j in range(n):
        model.addConstr(quicksum(y[i, j, k] for k in range(m)) <= c[i][j], name=f"compatibility_{i}_{j}")

# Each donor can donate to at most one patient: Σ y_ij <= 1, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j, k] for j in range(n) for k in range(m)) <= 1, name=f"donate_limit_{i}")

# Each patient can receive a kidney from at most one donor: Σ y_ij <= 1, ∀ j
for j in range(n):
    model.addConstr(quicksum(y[i, j, k] for i in range(n) for k in range(m)) <= 1, name=f"receive_limit_{j}")

# Reciprocal donation constraint: Σ y_ij = Σ y_ji, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j, k] for j in range(n) for k in range(m)) == quicksum(y[j, i, k] for j in range(n) for k in range(m)),
                    name=f"reciprocal_donation_{i}")

# Transplant Center Constrains
# use the living donor transplants in a year column as the transplant capacity
# the total transplants in this transplant center should be less than the capacity

# 4. Transplant center capacity
for k in range(m):
    capacity = centers.iloc[k]["Living Donor Transplants in a year"]
    model.addConstr(
        quicksum(y[i, j, k] for i in range(n) for j in range(n)) <= capacity,
        name=f"center_capacity_{k}"
    )

# Optimize the model
model.optimize()

# Print the results
print()
print("Results:")
transplants = []
if model.status == GRB.OPTIMAL:
    for i in range(n):
        for j in range(n):
            print(y[i,j,k].x)
            if y[i, j, k].x > 0.5:  # Only consider y[i,j] that are set to 1
                print(f"Donor {i} donates to Patient {j} at Center {k}")
                transplants.append((i, j))

print("Optimal transplants:", transplants)

**Model with Capacity (using data) (Jin's version)**

In [ ]:
# # Load data from CSV files
# pair_data_path = "./data/pair_small.csv"
# center_data_path = "./data/center_small.csv"

# # Load pair data
# pairs = pd.read_csv(pair_data_path)

# # Load transplant center data
# centers = pd.read_csv(center_data_path)

print("pairs", pairs)
print("centers", centers)

# Number of pairs and centers
n = len(pairs)
m = len(centers)

# Load the data
# data/pair_small.csv
# each pair have a state with its latitute and longitute
# we should connect pairs together to form a meaningful transplant
# the donor for a pair should match the patient of the matching pair
# data structure:
'''
dataset_id,Pair,Patient,Donor,Wife-P?,%Pra,Out-Deg,Altruist,State,lat,lng
1,1,A,B,0,0.05,2,0,Illinois,39.798363,-89.654961
1,2,O,A,0,0.05,4,0,California,38.576668,-121.493629
1,3,A,B,0,0.05,2,0,Michigan,42.733635,-84.555328
1,4,O,A,1,0.5875,3,0,California,38.576668,-121.493629
1,5,B,AB,0,0.05,0,0,New Jersey,40.220596,-74.769913
'''


# data/transplant_center_dataset.csv
'''
hdg,mix-text_weightBold,Deceased Donor Transplants in a year,Living Donor Transplants in a year,Getting a Deceased Donor Transplant Faster,1-Year kidney Survival,lat,lng
Mayo Clinic Hospital Arizona,"Phoenix, AZ",381,121,4,3,33.6594309,-111.9563162
Tampa General Hospital,"Tampa, FL",356,93,1,3,27.937527,-82.4587507
University of California San Francisco Medical Center,"San Francisco, CA",314,117,3,4,37.7627945,-122.4580139
Saint Barnabas Medical Center,"Livingston, NJ",282,105,5,2,40.782424,-74.315139
The Cleveland Clinic Foundation,"Cleveland, OH",280,46,4,4,41.5031026,-81.6200078
'''


# Create a model
model = Model("Kidney_Exchange_Capacity")

# Decision variables: y[i][j] = 1 if donor i donates to patient j, 0 otherwise
y = model.addVars(n, n, vtype=GRB.BINARY, name="y")  # Donor-to-patient match
c = model.addVars(m, vtype=GRB.BINARY, name="c")  # Transplant center selection

# pre-compute the travel costs
travel_costs = {}
for i in range(n):
    for j in range(n):
        # Example: travel_costs[(i, j)] = some_distance_function(pairs[i], pairs[j])
        pass


# Objective: Maximize the number of transplants and minimize travel distance between the selected transplant center and pairs
model.setObjective(
    quicksum(y[i, j] for i in range(n) for j in range(n)) -
    quicksum(travel_costs.get((i, j), 0) * y[i, j] for i in range(n) for j in range(n)),
    GRB.MAXIMIZE
)

# Another set of decision variables is c, which is a binary array with the length of the length of transplant center. it should only have one 1 as the selected transplant center

# Constraints
# Compatibility constraint: y_ij <= c_ij

# Each donor can donate to at most one patient: Σ y_ij <= 1, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for i in range(n)) <= 1, name=f"donate_limit_{i}")

# Each patient can receive a kidney from at most one donor: Σ y_ij <= 1, ∀ j
for j in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) <= 1, name=f"receive_limit_{j}")

# Reciprocal donation constraint: Σ y_ij = Σ y_ji, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) == quicksum(y[j, i] for j in range(n)),
                    name=f"reciprocal_donation_{i}")

# Transplant Center Constrains
# use the living donor transplants in a year column as the transplant capacity
# the total transplants in this transplant center should be less than the capacity

# Transplant center capacity constraint
for k in range(m):
    capacity = centers.iloc[k]["Living Donor Transplants in a year"]
    center_state = centers.iloc[k]["mix-text_weightBold"].split(", ")[1]  # Extract state
    model.addConstr(
        quicksum(
            y[i, j] for i in range(n) for j in range(n)
            if pairs.iloc[i]["State"] == center_state
        ) <= capacity * c[k],
        name=f"center_capacity_{k}"
    )

# Ensure only one transplant center is selected
model.addConstr(quicksum(c[k] for k in range(m)) == 1, name="single_center_selection")

# Optimize the model
model.optimize()


# print()
# print("Results: ")
# # Print the results
# transplants = []
# if model.status == GRB.OPTIMAL:
#     for i in range(n):
#         for j in range(n):
#             if y[i, j].x > 0.5:  # Only consider y[i,j] that are set to 1
#                 print(f"Donor {i} donated to Patient {j}")
#                 transplants.append((i, j))

# print("Optimal transplants:", transplants)

## Matching score system
the kidney transplant is not to find the exact match of blood type
We want to make the exact match have a higher score
The further distance match will receive a lower score
We want to maximize the matching score in our system


### **Kidney Transplant Blood Type Compatibility Table**

| **Donor Blood Type** | **Compatible Recipient Blood Types** | **Reason** |
|-----------------------|--------------------------------------|------------|
| **O**                | O, A, B, AB                         | Universal donor; no A/B antigens. |
| **A**                | A, AB                               | A antigen can only be accepted by A or AB. |
| **B**                | B, AB                               | B antigen can only be accepted by B or AB. |
| **AB**               | AB                                  | Universal recipient; can accept A, B, AB, or O. |

### Matching Scores

| **Donor → Recipient** | **Score** |
|------------------------|-----------|
| Exact match (e.g., O → O) | 10        |
| Compatible (e.g., O → A, A → AB) | 5        |
| Less ideal (e.g., O → AB) | 3        |
| Incompatible             | 0         |

In [ ]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

# # Load data from CSV files
# pair_data_path = "./data/pair_small.csv"
# center_data_path = "./data/center_small.csv"

# # Load pair data
# pairs = pd.read_csv(pair_data_path)

# # Load transplant center data
# centers = pd.read_csv(center_data_path)

print("pairs", pairs)
print("centers", centers)

# Number of pairs and centers
n = len(pairs)
m = len(centers)

# Load the data
# data/pair_small.csv
# each pair have a state with its latitute and longitute
# we should connect pairs together to form a meaningful transplant
# the donor for a pair should match the patient of the matching pair
# data structure:
'''
dataset_id,Pair,Patient,Donor,Wife-P?,%Pra,Out-Deg,Altruist,State,lat,lng
1,1,A,B,0,0.05,2,0,Illinois,39.798363,-89.654961
1,2,O,A,0,0.05,4,0,California,38.576668,-121.493629
1,3,A,B,0,0.05,2,0,Michigan,42.733635,-84.555328
1,4,O,A,1,0.5875,3,0,California,38.576668,-121.493629
1,5,B,AB,0,0.05,0,0,New Jersey,40.220596,-74.769913
'''


# data/transplant_center_dataset.csv
'''
hdg,mix-text_weightBold,Deceased Donor Transplants in a year,Living Donor Transplants in a year,Getting a Deceased Donor Transplant Faster,1-Year kidney Survival,lat,lng
Mayo Clinic Hospital Arizona,"Phoenix, AZ",381,121,4,3,33.6594309,-111.9563162
Tampa General Hospital,"Tampa, FL",356,93,1,3,27.937527,-82.4587507
University of California San Francisco Medical Center,"San Francisco, CA",314,117,3,4,37.7627945,-122.4580139
Saint Barnabas Medical Center,"Livingston, NJ",282,105,5,2,40.782424,-74.315139
The Cleveland Clinic Foundation,"Cleveland, OH",280,46,4,4,41.5031026,-81.6200078
'''


# Create a model
model = Model("Kidney_Exchange_Capacity")

# Decision variables: y[i][j] = 1 if donor i donates to patient j, 0 otherwise
y = model.addVars(n, n, vtype=GRB.BINARY, name="y")  # Donor-to-patient match
c = model.addVars(m, vtype=GRB.BINARY, name="c")  # Transplant center selection

# pre-compute the travel costs
travel_costs = {}
for i in range(n):
    for j in range(n):
        # Example: travel_costs[(i, j)] = some_distance_function(pairs[i], pairs[j])
        pass


# Objective: Maximize the number of transplants and minimize travel distance between the selected transplant center and pairs
model.setObjective(
    quicksum(y[i, j] for i in range(n) for j in range(n)) -
    quicksum(travel_costs.get((i, j), 0) * y[i, j] for i in range(n) for j in range(n)),
    GRB.MAXIMIZE
)

# Another set of decision variables is c, which is a binary array with the length of the length of transplant center. it should only have one 1 as the selected transplant center

# Constraints
# Compatibility constraint: y_ij <= c_ij

# Introducing a matching score system
# Define matching scores based on donor-patient compatibility
matching_scores = {}
blood_type_scores = {
    ('O', 'O'): 10, ('O', 'A'): 5, ('O', 'B'): 5, ('O', 'AB'): 3,
    ('A', 'A'): 10, ('A', 'AB'): 5,
    ('B', 'B'): 10, ('B', 'AB'): 5,
    ('AB', 'AB'): 10
}

for i in range(n):
    for j in range(n):
        donor = pairs.iloc[i]["Donor"]
        patient = pairs.iloc[j]["Patient"]
        matching_scores[(i, j)] = blood_type_scores.get((donor, patient), 0)

# Update the objective function to maximize matching scores
model.setObjective(
    quicksum(matching_scores[i, j] * y[i, j] for i in range(n) for j in range(n)) -
    quicksum(travel_costs.get((i, j), 0) * y[i, j] for i in range(n) for j in range(n)),
    GRB.MAXIMIZE
)


# Each donor can donate to at most one patient: Σ y_ij <= 1, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for i in range(n)) <= 1, name=f"donate_limit_{i}")

# Each patient can receive a kidney from at most one donor: Σ y_ij <= 1, ∀ j
for j in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) <= 1, name=f"receive_limit_{j}")

# Reciprocal donation constraint: Σ y_ij = Σ y_ji, ∀ i
for i in range(n):
    model.addConstr(quicksum(y[i, j] for j in range(n)) == quicksum(y[j, i] for j in range(n)),
                    name=f"reciprocal_donation_{i}")

# Transplant Center Constrains
# use the living donor transplants in a year column as the transplant capacity
# the total transplants in this transplant center should be less than the capacity

# Transplant center capacity constraint
for k in range(m):
    capacity = centers.iloc[k]["Living Donor Transplants in a year"]
    center_state = centers.iloc[k]["mix-text_weightBold"].split(", ")[1]  # Extract state
    model.addConstr(
        quicksum(
            y[i, j] for i in range(n) for j in range(n)
            if pairs.iloc[i]["State"] == center_state
        ) <= capacity * c[k],
        name=f"center_capacity_{k}"
    )

# Ensure only one transplant center is selected
model.addConstr(quicksum(c[k] for k in range(m)) == 1, name="single_center_selection")

# Optimize the model
model.optimize()


# print()
# print("Results: ")
# # Print the results
# transplants = []
# if model.status == GRB.OPTIMAL:
#     for i in range(n):
#         for j in range(n):
#             if y[i, j].x > 0.5:  # Only consider y[i,j] that are set to 1
#                 print(f"Donor {i} donated to Patient {j}")
#                 transplants.append((i, j))

# print("Optimal transplants:", transplants)

**Model with Transportation Distance (using data)**

In [ ]:

# Number of pairs and centers
n = len(pairs)
m = len(centers)


In [ ]:
# Create a model
model = Model("Kidney_Exchange_Capacity")

# Decision variables
x = model.addVars(n, n, m, vtype=GRB.BINARY, name="x")  # x[i,j,k] = 1 if donor i donates to patient j through center k
y = model.addVars(m, vtype=GRB.BINARY, name="y")  # y[k] = 1 if center k is used

# Calculate distances
distances = {}
for i in range(n):
    for k in range(m):
        pair_coords = (pairs.iloc[i]["lat"], pairs.iloc[i]["lng"])
        center_coords = (centers.iloc[k]["lat"], centers.iloc[k]["lng"])
        distances[(i, k)] = geodesic(pair_coords, center_coords).kilometers

# Compatibility matrix
comp = {}
for i in range(n):
    for j in range(n):
        donor = pairs.iloc[i]["Donor"]
        patient = pairs.iloc[j]["Patient"]
        comp[(i, j)] = 1 if (donor, patient) in [('O', 'O'), ('O', 'A'), ('O', 'B'), ('O', 'AB'),
                                                 ('A', 'A'), ('A', 'AB'),
                                                 ('B', 'B'), ('B', 'AB'),
                                                 ('AB', 'AB')] else 0

# Parameter
alpha = 0.0001  # Adjust this value to balance exchanges vs. travel distance

# Objective function
model.setObjective(
    quicksum(x[i, j, k] for i in range(n) for j in range(n) for k in range(m)) -
    alpha * quicksum((distances[i, k] + distances[j, k]) * x[i, j, k] for i in range(n) for j in range(n) for k in range(m)),
    GRB.MAXIMIZE
)

# Constraints
# 1. Donor limit
for i in range(n):
    model.addConstr(quicksum(x[i, j, k] for j in range(n) for k in range(m)) <= 1, name=f"donor_limit_{i}")

# 2. Patient limit
for j in range(n):
    model.addConstr(quicksum(x[i, j, k] for i in range(n) for k in range(m)) <= 1, name=f"patient_limit_{j}")

# 3. Compatibility constraint
for i in range(n):
    for j in range(n):
        for k in range(m):
            model.addConstr(x[i, j, k] <= comp[i, j], name=f"compatibility_{i}_{j}_{k}")

# 4. Transplant center capacity
for k in range(m):
    capacity = centers.iloc[k]["Living Donor Transplants in a year"]
    model.addConstr(
        quicksum(x[i, j, k] for i in range(n) for j in range(n)) <= capacity * y[k],
        name=f"center_capacity_{k}"
    )

# 5. Ensure y[k] is 1 if center k is used
for k in range(m):
    model.addConstr(
        y[k] >= quicksum(x[i, j, k] for i in range(n) for j in range(n)) / (n * n),
        name=f"center_used_{k}"
    )

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("\nOptimal Solution Found:")
    for i in range(n):
        for j in range(n):
            for k in range(m):
                if x[i, j, k].x > 0.5:
                    print(f"Donor {i} donates to Patient {j} through Center {k}")

    used_centers = [k for k in range(m) if y[k].x > 0.5]
    print(f"\nSelected Transplant Centers: {[centers.iloc[k]['mix-text_weightBold'] for k in used_centers]}")

    total_exchanges = sum(x[i, j, k].x for i in range(n) for j in range(n) for k in range(m))
    total_travel_distance = sum((distances[i, k] + distances[j, k]) * x[i, j, k].x for i in range(n) for j in range(n) for k in range(m))
    print(f"\nTotal Exchanges: {total_exchanges}")
    print(f"Total Travel Distance: {total_travel_distance:.2f} km")
    print(f"Objective Value: {model.objVal:.2f}")
else:
    print("No optimal solution found.")

# Combined Model


---

## **Kidney Exchange Optimization Model**

### **Sets**
- $I$: Set of donor-recipient pairs ($ i, j \in I $, where $ i \neq j $)
- $ K $: Set of transplant centers ($ k \in K $)

### **Parameters**
- $ d_{ik} $: Distance (in kilometers) from donor-recipient pair $ i $ to transplant center $ k $
- $ d_{jk} $: Distance (in kilometers) from donor-recipient pair $ j $ to transplant center $ k $
- $c_k $: Capacity of transplant center $ k $ (maximum number of transplants it can handle)
- $comp_{ij} $: Compatibility between donor $ i $ and recipient $ j $ ($1 $ if compatible, $ 0 $ otherwise)
- $ n $: Total number of donor-recipient pairs
- $ m $: Total number of transplant centers
- $ \alpha > 0 $: Weight factor to balance maximizing exchanges and minimizing travel distance

### **Decision Variables**
- $ x_{ijk} \in \{0,1\} $: Binary variable, equals 1 if donor $ i $ donates to recipient $ j $ through transplant center $ k $, and 0 otherwise
- $y_k \in \{0,1\} $: Binary variable, equals 1 if transplant center $k $ is used for any exchange, and 0 otherwise

---

### **Objective Function**

Maximize:
$$
\sum_{i=1}^n \sum_{j=1}^n \sum_{k=1}^m x_{ijk} -
\alpha \cdot
\sum_{i=1}^n
\sum_{j=1}^n
\sum_{k=1}^m
(d_{ik} + d_{jk})
\cdot x_{ijk}
$$

**Explanation**:
- The first term maximizes the total number of kidney exchanges.
- The second term minimizes the total travel distance for all selected exchanges, weighted by the parameter $$ \alpha > 0$$.

---

### **Constraints**

#### 1. Donor Limit:
Each donor can donate to at most one recipient:
$$
\sum_{j=1}^n
\sum_{k=1}^m
x_{ijk}
\leq 1,
\quad
\forall i = 1, 2, ..., n
$$

#### 2. Recipient Limit:
Each recipient can receive a kidney from at most one donor:
$$
\sum_{i=1}^n
\sum_{k=1}^m
x_{ijk}
\leq 1,
\quad
\forall j = 1, 2, ..., n
$$

#### 3. Reciprocal Donation Constraint (Individual Rationality):
Donor in pair i will donate if and only if their patient in pair i receives a kidney.
$$
\sum_{j=1}^n
\sum_{k=1}^m
x_{ijk}
= \sum_{j=1}^n
\sum_{k=1}^m
x_{jik},
\quad
\forall i = 1, 2, ..., n
$$

#### 4. Compatibility Constraint:
A match between donor $$ i $$ and recipient $$ j $$ is only allowed if they are compatible:
$$
x_{ijk}
\leq comp_{ij},
\quad
\forall i, j = 1, ..., n,
\, k = 1, ..., m
$$

#### 5. Transplant Center Capacity (Ashwini):
The total number of transplants performed at a center cannot exceed its capacity:
$$
\sum_{i=1}^n
\sum_{j=1}^n
x_{ijk}
\leq c_k,
\quad
\forall k = 1, ..., m
$$

#### 5. Transplant Center Capacity:
The total number of transplants performed at a center cannot exceed its capacity:
$$
\sum_{i=1}^n
\sum_{j=1}^n
x_{ijk}
\leq c_k
\cdot y_k,
\quad
\forall k = 1, ..., m
$$

#### 6. Center Usage Constraint:
$$
y_k
\geq
\frac{1}{n^2}
\cdot
\sum_{i=1}^n
\sum_{j=1}^n x_{ijk},
\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,\,  
k = 1, ..., m
$$

#### 7. Binary Constraints:
All decision variables are binary:
$$
x_{ijk}, y_k \in \{0, 1\},
\,\,\,\,\,  
i,j = 1,...,n;  
k = 1,...,m
$$

---


In [ ]:
import pandas as pd
from gurobipy import Model, GRB, quicksum
from geopy.distance import geodesic

#SAMPLE SIZE:
pair_size = 10
center_size = 5

# Load data from CSV files
pair_data_path = "./data/kidney_data_with_states.csv"
center_data_path = "./data/transplant_center_dataset.csv"

# Load pair data and take a subset
pairs = pd.read_csv(pair_data_path)
pairs = pairs.sample(n=min(pair_size, len(pairs)), random_state=42)

# Load transplant center data
centers = pd.read_csv(center_data_path)
centers = centers.sample(n=min(center_size, len(pairs)), random_state=42)
# Reset index for pairs DataFrame
pairs = pairs.reset_index(drop=True)

n = len(pairs)
m = len(centers)

print(f"Number of pairs: {n}")
print(f"Number of centers: {m}")

# Continue with the rest of your optimization model...
centers = pd.read_csv(center_data_path)

In [ ]:
# Create a model
model = Model("Kidney_Exchange_Capacity")

# Decision variables
x = model.addVars(n, n, m, vtype=GRB.BINARY, name="x")  # x[i,j,k] = 1 if donor i donates to patient j through center k
y = model.addVars(m, vtype=GRB.BINARY, name="y")  # y[k] = 1 if center k is used

# Calculate distances
distances = {}
for i in range(n):
    for k in range(m):
        pair_coords = (pairs.iloc[i]["lat"], pairs.iloc[i]["lng"])
        center_coords = (centers.iloc[k]["lat"], centers.iloc[k]["lng"])
        distances[(i, k)] = geodesic(pair_coords, center_coords).kilometers

# Compatibility matrix
comp = {}
for i in range(n):
    for j in range(n):
        donor = pairs.iloc[i]["Donor"]
        patient = pairs.iloc[j]["Patient"]
        comp[(i, j)] = 1 if (donor, patient) in [('O', 'O'), ('O', 'A'), ('O', 'B'), ('O', 'AB'),
                                                 ('A', 'A'), ('A', 'AB'),
                                                 ('B', 'B'), ('B', 'AB'),
                                                 ('AB', 'AB')] else 0

# Parameter
alpha = 0.0001  # Adjust this value to balance exchanges vs. travel distance

# Objective function
model.setObjective(
    quicksum(x[i, j, k] for i in range(n) for j in range(n) for k in range(m)) -
    alpha * quicksum((distances[i, k] + distances[j, k]) * x[i, j, k] for i in range(n) for j in range(n) for k in range(m)),
    GRB.MAXIMIZE
)

# Constraints
# 1. Donor limit
for i in range(n):
    model.addConstr(quicksum(x[i, j, k] for j in range(n) for k in range(m)) <= 1, name=f"donor_limit_{i}")

# 2. Patient limit
for j in range(n):
    model.addConstr(quicksum(x[i, j, k] for i in range(n) for k in range(m)) <= 1, name=f"patient_limit_{j}")

# 3. Reciprocal Donation Constraint
for i in range(n):
    model.addConstr(quicksum(x[i, j, k] for j in range(n) for k in range(m)) == quicksum(x[j, i, k] for j in range(n) for k in range(m)), name=f"donor_limit_{i}")

# 3. Compatibility constraint
for i in range(n):
    for j in range(n):
        for k in range(m):
            model.addConstr(x[i, j, k] <= comp[i, j], name=f"compatibility_{i}_{j}_{k}")

# 4. Transplant center capacity
for k in range(m):
    capacity = centers.iloc[k]["Living Donor Transplants in a year"]
    model.addConstr(
        quicksum(x[i, j, k] for i in range(n) for j in range(n)) <= capacity * y[k],
        name=f"center_capacity_{k}"
    )

# 5. Ensure y[k] is 1 if center k is used
for k in range(m):
    model.addConstr(
        y[k] >= quicksum(x[i, j, k] for i in range(n) for j in range(n)) / (n * n),
        name=f"center_used_{k}"
    )

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print("\nOptimal Solution Found:")
    for i in range(n):
        for j in range(n):
            for k in range(m):
                if x[i, j, k].x > 0.5:
                    print(f"Donor {i} donates to Patient {j} through Center {k}")

    used_centers = [k for k in range(m) if y[k].x > 0.5]
    print(f"\nSelected Transplant Centers: {[centers.iloc[k]['mix-text_weightBold'] for k in used_centers]}")

    total_exchanges = sum(x[i, j, k].x for i in range(n) for j in range(n) for k in range(m))
    total_travel_distance = sum((distances[i, k] + distances[j, k]) * x[i, j, k].x for i in range(n) for j in range(n) for k in range(m))
    print(f"\nTotal Exchanges: {total_exchanges}")
    print(f"Total Travel Distance: {total_travel_distance:.2f} km")
    print(f"Objective Value: {model.objVal:.2f}")
else:
    print("No optimal solution found.")

# Visualize the Result

In [ ]:
import folium
from folium import FeatureGroup, LayerControl
from IPython.display import display

# Extract the results
transplants = []
if model.status == GRB.OPTIMAL:
    for i in range(n):
        for j in range(n):
            if y[i, j].x > 0.5:  # Only consider y[i,j] that are set to 1
                transplants.append((i, j))

# Create a base map with a simplistic style
map_center = [39.8283, -98.5795]  # Center of the US
map_transplants = folium.Map(location=map_center, zoom_start=5, tiles="CartoDB Positron")

# Feature groups for toggling visibility
donors_group = FeatureGroup(name="Donors", show=True)
recipients_group = FeatureGroup(name="Recipients", show=True)
centers_group = FeatureGroup(name="Transplant Centers", show=True)
connections_group = FeatureGroup(name="Donor-Recipient Connections", show=True)
center_connections_group = FeatureGroup(name="Pair-to-Center Connections", show=True)

# Add donors to the map
for i in range(n):
    donor_info = (f"Donor {i}: {pairs.iloc[i]['Donor']} from {pairs.iloc[i]['State']}<br>"
                  f"Latitude: {pairs.iloc[i]['lat']}, Longitude: {pairs.iloc[i]['lng']}")
    folium.CircleMarker(
        location=[pairs.iloc[i]["lat"], pairs.iloc[i]["lng"]],
        radius=6,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip=donor_info
    ).add_to(donors_group)

# Add recipients to the map
for i in range(n):
    recipient_info = (f"Recipient {i}: {pairs.iloc[i]['Patient']} in {pairs.iloc[i]['State']}<br>"
                      f"Latitude: {pairs.iloc[i]['lat']}, Longitude: {pairs.iloc[i]['lng']}")
    folium.CircleMarker(
        location=[pairs.iloc[i]["lat"], pairs.iloc[i]["lng"]],
        radius=6,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip=recipient_info
    ).add_to(recipients_group)

# Add transplant centers to the map
for k in range(m):
    center_name = centers.iloc[k]["hdg"]
    center_location = centers.iloc[k]["mix-text_weightBold"]
    center_info = (f"Center {k}: {center_name}<br>"
                   f"Location: {center_location}<br>"
                   f"Capacity: {centers.iloc[k]['Living Donor Transplants in a year']}<br>"
                   f"Latitude: {centers.iloc[k]['lat']}, Longitude: {centers.iloc[k]['lng']}")
    folium.CircleMarker(
        location=[centers.iloc[k]["lat"], centers.iloc[k]["lng"]],
        radius=8,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.6,
        tooltip=center_info
    ).add_to(centers_group)

# Highlight the selected transplant center
selected_center_idx = None
for k in range(m):
    if c[k].x > 0.5:
        selected_center_idx = k
        center_name = centers.iloc[k]["hdg"]
        center_location = centers.iloc[k]["mix-text_weightBold"]
        center_info = (f"Selected Center {k}: {center_name}<br>"
                       f"Location: {center_location}<br>"
                       f"Latitude: {centers.iloc[k]['lat']}, Longitude: {centers.iloc[k]['lng']}")
        folium.CircleMarker(
            location=[centers.iloc[k]["lat"], centers.iloc[k]["lng"]],
            radius=10,
            color='orange',
            fill=True,
            fill_color='orange',
            fill_opacity=0.8,
            tooltip=center_info
        ).add_to(centers_group)

# Draw lines for donor-recipient transplants
for donor, recipient in transplants:
    donor_info = (f"Donor {donor}: {pairs.iloc[donor]['Donor']} from {pairs.iloc[donor]['State']}")
    recipient_info = (f"Recipient {recipient}: {pairs.iloc[recipient]['Patient']} in {pairs.iloc[recipient]['State']}")
    folium.PolyLine(
        locations=[
            [pairs.iloc[donor]["lat"], pairs.iloc[donor]["lng"]],
            [pairs.iloc[recipient]["lat"], pairs.iloc[recipient]["lng"]]
        ],
        color='purple',
        weight=2,
        opacity=0.7,
        tooltip=f"Match: {donor_info} → {recipient_info}"
    ).add_to(connections_group)

# Draw lines connecting recipients to the transplant center
if selected_center_idx is not None:
    for _, recipient in transplants:
        recipient_info = (f"Recipient {recipient}: {pairs.iloc[recipient]['Patient']} in {pairs.iloc[recipient]['State']}")
        center_info = centers.iloc[selected_center_idx]["hdg"]
        folium.PolyLine(
            locations=[
                [pairs.iloc[recipient]["lat"], pairs.iloc[recipient]["lng"]],
                [centers.iloc[selected_center_idx]["lat"], centers.iloc[selected_center_idx]["lng"]]
            ],
            color='cyan',
            weight=2,
            opacity=0.7,
            tooltip=f"Connection: {recipient_info} → Center: {center_info}"
        ).add_to(center_connections_group)

# Add all feature groups to the map
donors_group.add_to(map_transplants)
recipients_group.add_to(map_transplants)
centers_group.add_to(map_transplants)
connections_group.add_to(map_transplants)
center_connections_group.add_to(map_transplants)

# Add layer control to toggle visibility
LayerControl().add_to(map_transplants)

# Display the map inline in Colab
display(map_transplants)


In [ ]:
import pandas as pd
import folium
from folium import FeatureGroup, LayerControl, plugins

# Load the data
centers = pd.read_csv("results/centers_usage_smalldata_transport.csv")
matches = pd.read_csv("results/kidney_matches_smalldata_transport.csv")

# Create a base map centered in the US
map_center = [39.8283, -98.5795]
map_transplants = folium.Map(location=map_center, zoom_start=5, tiles="CartoDB Positron")

# Define colors for each center (7 distinct colors)
center_colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown', 'pink']

# Create feature groups for each center
center_groups = {}
for idx, center in centers.iterrows():
    if center['Is_Used']:
        color = center_colors[idx]
        group = FeatureGroup(name=f"Center {center['Center_ID']} Group")

        # Add center marker (square)
        folium.RegularPolygonMarker(
            location=[center['Lat'], center['Lng']],
            number_of_sides=4,
            radius=8,
            color=color,
            fill=True,
            popup=f"Center {center['Center_ID']}: {center['Name']}"
        ).add_to(group)

        # Add donors and recipients for this center
        center_matches = matches[matches['Center_ID'] == center['Center_ID']]
        for _, match in center_matches.iterrows():
            # Add donor (circle)
            folium.CircleMarker(
                location=[match['Donor_Lat'], match['Donor_Lng']],
                radius=6,
                color=color,
                fill=True,
                popup=f"Donor {match['Donor_ID']}"
            ).add_to(group)

            # Add recipient (triangle)
            folium.RegularPolygonMarker(
                location=[match['Recipient_Lat'], match['Recipient_Lng']],
                number_of_sides=3,
                radius=6,
                color=color,
                fill=True,
                popup=f"Recipient {match['Recipient_ID']}"
            ).add_to(group)

            # Add directed arrow from donor to recipient
            plugins.AntPath(
                locations=[[match['Donor_Lat'], match['Donor_Lng']],
                          [match['Recipient_Lat'], match['Recipient_Lng']]],
                color=color,
                weight=2,
                popup=f"Donor {match['Donor_ID']} → Recipient {match['Recipient_ID']}"
            ).add_to(group)

        center_groups[center['Center_ID']] = group
        group.add_to(map_transplants)

# Add layer control
LayerControl().add_to(map_transplants)


display(map_transplants)
# Save the map
map_transplants.save("kidney_exchanges_map.html")